In [13]:
import folium
from folium.plugins import MarkerCluster
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
from folium.plugins import MarkerCluster
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests


sensebox_id = "67cac102d2a4eb00071d6ac9"

In [ ]:
def get_all_locations(sensebox_id):
    url = f"https://api.opensensemap.org/boxes/{sensebox_id}/locations"
    params = {
        "from-date": "2015-01-01T00:00:00Z",
        "format": "json"
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Fehler beim Abrufen der Daten: {response.status_code}")

    data = response.json()

    locations = []
    for entry in data:
        timestamp = entry.get("timestamp")
        coordinates = entry.get("coordinates")
        if timestamp and coordinates:
            locations.append((timestamp, coordinates))

    return locations

orte = get_all_locations(sensebox_id)

[('2025-03-07T09:48:50.961Z', [104.994063, 77.585865, 2e+31]),
 ('2025-03-07T09:49:38.430Z', [104.994063, 77.585865, 0])]

In [38]:
def show_colored_map(locations):
    if not locations:
        print("Keine Koordinaten vorhanden.")
        return

    # Sortieren nach Zeit
    locations_sorted = sorted(locations, key=lambda x: x[0])
    timestamps = [datetime.fromisoformat(t[0].replace("Z", "+00:00")) for t in locations_sorted]

    # Farbskala vorbereiten
    norm = colors.Normalize(vmin=0, vmax=len(timestamps) - 1)
    colormap = cm.get_cmap("plasma")
    colors_hex = [colors.to_hex(colormap(norm(i))) for i in range(len(timestamps))]

    # Karte auf ersten Punkt zentrieren
    first_coord = locations_sorted[0][1]
    lat = first_coord[1]
    lon = first_coord[0]
    fmap = folium.Map(location=[lat, lon], zoom_start=4)
    marker_cluster = MarkerCluster().add_to(fmap)

    # Marker hinzufügen
    for (timestamp, coord), color in zip(locations_sorted, colors_hex):
        lon = coord[0]
        lat = coord[1]
        alt = coord[2] if len(coord) > 2 else None
        popup_text = f"{timestamp}"
        if alt is not None:
            popup_text += f"<br>Höhe: {alt:.2f} m"

        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            popup=popup_text,
            color=color,
            fill=True,
            fill_color=color
        ).add_to(marker_cluster)

    return fmap  # ← Jupyter zeigt dies automatisch

map = show_colored_map(orte)

C:\Users\tobia\AppData\Local\Temp\ipykernel_10368\1465362382.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap("plasma")
